In [ ]:
from itertools import product

In [ ]:
%ntbl pull datasets "hh_ru/hh_ru_dataset.csv"

In [ ]:
df = pd.read_csv('../datasets/hh_ru/hh_ru_dataset.csv')

In [ ]:
df_cleared = df.copy()

Исследуемые фильтры:
1. по final_state (удаление response) - fs
2. по dropna (education_level, initial_state, final_state, work_schedule, compensation_from, compensation_to) - na
3. нижний порог expected_salary - exp
4. пороги compensation_to - comp

Сначала применяем все 100% нужные фильтры

In [ ]:
before = len(df_cleared)
df_cleared = df_cleared[(df_cleared['year_of_birth'] >= 1953) & (df_cleared['year_of_birth'] <= 2007)]
after = len(df_cleared)
before - after

In [ ]:
before = len(df_cleared)
df_cleared = df_cleared.drop_duplicates(['topic_id'])
after = len(df_cleared)
before - after

In [ ]:
before = len(df_cleared)
df_cleared = df_cleared[df_cleared['compensation_from'] <= df_cleared['compensation_to']]
after = len(df_cleared)
before - after

In [ ]:
before = len(df_cleared)
df_cleared = df_cleared[(2023 - df_cleared['work_experience_months'] / 12 - df_cleared['year_of_birth'] >= 16) & (df_cleared['work_experience_months'] <= 600)]
after = len(df_cleared)
before - after

In [ ]:
quantile_param = 0.03
quantiles = {}
for prof in df['profession'].unique():
    df_prof = df[df['profession'] == prof]
    quantiles[prof] = (df_prof['expected_salary'].quantile(quantile_param),
                       df_prof['expected_salary'].quantile(1 - quantile_param))

df_cleared.loc[:,'exp_salary_min_limit'] = [quantiles[i][0] for i in df_cleared['profession']]
df_cleared.loc[:,'exp_salary_max_limit'] = [quantiles[i][1] for i in df_cleared['profession']]

# проверка по ожидаемой зарплате
before = len(df_cleared)
df_cleared = df_cleared[df_cleared['expected_salary'] <= df_cleared['exp_salary_max_limit']]
after = len(df_cleared)
before - after

Приступаем к необязательным

In [ ]:
results = pd.DataFrame(columns=('fs', 'na', 'exp', 'comp', 'final_count'))

In [ ]:
quantiles_exp = {}
for prof in df['profession'].unique():
    df_prof = df[df['profession'] == prof]
    quantiles_exp[prof] = (df_prof['compensation_to'].quantile(quantile_param),
                       df_prof['compensation_to'].quantile(1 - quantile_param))
    
df_cleared.loc[:,'compensation_to_min_limit'] = [quantiles_exp[i][0] for i in df_cleared['profession']]
df_cleared.loc[:,'compensation_to_max_limit'] = [quantiles_exp[i][1] for i in df_cleared['profession']]

# проверка по compensation_to
df_cleared = df_cleared[(df_cleared['compensation_to'] >= df_cleared['compensation_to_min_limit']) & 
                            (df_cleared['compensation_to'] <= df_cleared['compensation_to_max_limit'])]


In [ ]:
def fs(df):
    local_df = df.copy()
    before = len(local_df)
    local_df = local_df[-((local_df.final_state == 'response') | (local_df.final_state.isna()))]
    after = len(local_df)
    return local_df, before - after


def na(df):
    local_df = df.copy()
    before = len(local_df)
    local_df = local_df.dropna(axis=0, subset=['education_level', 'initial_state', 'work_schedule', 'compensation_from', 'compensation_to', 'final_state'])
    after = len(local_df)
    return local_df, before - after


def exp(df):
    global quantiles
    local_df = df.copy()
    before = len(local_df)
    local_df = local_df[local_df['expected_salary'] >= local_df['exp_salary_min_limit']]
    after = len(local_df)
    return local_df, before - after


def comp(df):
    global quantiles
    local_df = df.copy()
    before = len(local_df)
    local_df = local_df[(local_df['compensation_to'] >= local_df['compensation_to_min_limit'])& (local_df['compensation_to'] <= local_df['compensation_to_max_limit'])]
    after = len(local_df)
    return local_df, before - after

In [ ]:
funcs = (fs, na, exp, comp)

In [ ]:
for flags in product((False, True), repeat=4):
    temp_df = df_cleared.copy()
    row = []
    for flag, func in zip(flags, funcs):
        deleted_rows = -1
        if flag:
            temp_df, deleted_rows = func(temp_df)
        row.append(deleted_rows)
    row.append(len(temp_df))
    results.loc[len(results.index)] = row
            
    
    

In [ ]:
results

In [ ]:
results.to_excel('исследование_спорных_фильтров.xlsx')